# k-Means Algorithmus

Die Aufgabe ist, gegebene Daten in möglichst ähnliche Cluster zu segmentieren.

"k" bezieht sich dabei auf die Anzahl der Cluster, die man erhalten möchte.

Zunächst mal importieren wir ein paar Bibliotheken:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()  # for plot styling
import numpy as np

plt.rcParams["figure.figsize"] = (10,8)

## Grobe Beschreibung

Der k-Means-Algorithmus sucht nach einer vorgegebenen Anzahl von Clustern innerhalb eines unmarkierten mehrdimensionalen Datensatzes.
Dazu verwendet er eine einfache Vorstellung davon, wie das optimale Clustering aussieht:

-  Das "Clusterzentrum" ist das arithmetische Mittel aller zum Cluster gehörenden Punkte.
-  Jeder Punkt liegt näher an seinem eigenen Clusterzentrum als an anderen Clusterzentren.

Diese beiden Annahmen bilden die Grundlage des k-Means-Algorithmus. Wir werden bald genau darauf
eingehen, wie der Algorithmus diese Lösung erreicht, aber sehen wir uns zunächst einen einfachen
Datensatz an und sehen uns das Ergebnis des k-Means an.

## Unsere Daten
Zunächst erzeugen wir uns einen Datensatz:

In [ ]:
from sklearn.datasets import make_blobs

X, y_true = make_blobs(n_samples=300, centers=4,
                       cluster_std=0.60, random_state=0)
sns.scatterplot(X[:, 0], X[:, 1]);

Man sieht schon mit bloßem Auge vier Cluster.

## Fertiger Algorithmus in Scikit-Learn

Wir nutzen nun den k-Means Algorithmus von Scikit-learn. Das Muster ist dabei immer gleich:

- zunächst wählt man ein Modell
- und ruft dann die `.fit`-Methode, um das Modell zu trainieren
- schließlich erzeugt man mit der `.predict`-Methode die Ergebnisdaten


In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=4)
kmeans.fit(X)
y_kmeans = kmeans.predict(X)

Wir können das Ergebnis visualisieren - die großen roten Punkte zeigen die Zenter der Cluster an:

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=y_kmeans, s=50, cmap='viridis')

centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='red', s=100);




## Implementierung

Der Algorithmus ist einfach genug, dass wir ihn in wenigen Zeilen selbst schreiben können:

In [ ]:
from sklearn.metrics import pairwise_distances_argmin

def find_clusters(X, n_clusters, rseed=2):
    # 1. wähle zufällige Zentren:
    rng = np.random.default_rng(rseed)
    i = rng.permutation(X.shape[0])[:n_clusters]
    centers = X[i]

    while True:
        # Färbe entsprechend dem nächtsgelegenen Zentrum ein
        labels = pairwise_distances_argmin(X, centers)

        # Ermittle neue Zentren als Mittelpunkt des Clusters
        new_centers = np.array([X[labels == i].mean(0) for i in range(n_clusters)])

        # Prüfe, ob fertig
        if np.all(centers == new_centers):
            break
        centers = new_centers

    return centers, labels

centers, labels = find_clusters(X, 4)

Schauen wir uns das Ergebnis an:

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=labels, s=50, cmap='viridis')
plt.scatter(centers[:, 0], centers[:, 1], c='red', s=100);


Wir können uns das auch interaktiv anschauen:

In [ ]:
from ipywidgets import interact
from sklearn.metrics import pairwise_distances_argmin
import ipywidgets as widgets

def plot_kmeans_interactive(min_clusters=1, max_clusters=6):
    def plot_points(X, labels, n_clusters):
        plt.scatter(X[:, 0], X[:, 1], c=labels, s=50, cmap='viridis', vmin=0, vmax=n_clusters - 1)

    def plot_centers(centers):
        plt.scatter(centers[:, 0], centers[:, 1], marker='o',
                    c=np.arange(centers.shape[0]),
                    s=200, cmap='viridis')
        plt.scatter(centers[:, 0], centers[:, 1], marker='o', c='red', s=100)

    def _kmeans_step(frame=0, n_clusters=4, rseed=2):
        labels = np.zeros(X.shape[0])
        rng = np.random.default_rng(rseed)
        i = rng.permutation(X.shape[0])[:n_clusters]
        centers = X[i]

        nsteps = frame // 3

        for i in range(nsteps + 1):
            old_centers = centers
            if i < nsteps or frame % 3 > 0:
                labels = pairwise_distances_argmin(X, centers)

            if i < nsteps or frame % 3 > 1:
                centers = np.array([X[labels == j].mean(0)
                                    for j in range(n_clusters)])
                nans = np.isnan(centers)
                centers[nans] = old_centers[nans]

        # plot the data and cluster centers
        plot_points(X, labels, n_clusters)
        plot_centers(old_centers)

        # plot new centers if third frame
        if frame % 3 == 2:
            for i in range(n_clusters):
                plt.annotate('', centers[i], old_centers[i],
                             arrowprops=dict(arrowstyle='->', linewidth=3, color='yellow'))
            plot_centers(centers)

        plt.xlim(-4, 4)
        plt.ylim(-2, 10)

        if frame == 0:
            plt.title("Setze die Zentren zufällig")
        elif frame % 3 == 1:
            plt.title("1. Färbe die Punkte entsprechend dem nächsten Zentrum ein")
        elif frame % 3 == 2:
            plt.title("2. Setze die Zentren auf den Cluster-Durchschnitt")
        elif frame % 3 == 0 and frame > 0:
            plt.title("Neue Positionen der Zentren")

    return interact(_kmeans_step, frame=widgets.IntSlider(min=0, max=100, step=1, value=0),
                    n_clusters=[1, 2, 3, 4, 5, 6, 15],
                    rseed=[2,4])

plot_kmeans_interactive();


## Limitierungen des k-Means Algorithmus

### Die Anzahl der Cluster muss im Vorfeld gewählt werden

Wie man sieht ist die Wahl von k ganz entscheidend für eine saubere Trennung der Cluster.

- Zu kleines k differenziert die Cluster ggf. nicht ausreichend.
- Zu großes k trennt ggf. willkürlich Cluster auf.

### Lösung ist nicht unbedingt die global beste Lösung

Das Verfahren konvergiert zwar garantiert, allerdings nicht notwendigerweise zur global optimalen Lösung.

Hier wirkt sich ganz entscheidend die anfängliche Anordnung der Zentren aus. Daher wird die Berechnung
in der Praxis mehrere Male mit jeweils zufällig gewählten Anfangszentren wiederholt.

Scikit-Learn macht das defaultmäßig 10 mal, gesteuert über den Parameter `n_init`.

In [ ]:
centers, labels = find_clusters(X, n_clusters=4, rseed=4)
plt.scatter(X[:, 0], X[:, 1], c=labels, s=50, cmap='viridis')
plt.scatter(centers[:, 0], centers[:, 1], c='red', s=100, alpha=0.5);

### Langsam, bei großen Datensets

Da jede Iteration von k-Means auf jeden Punkt im Datensatz zugreifen muss, kann der Algorithmus
relativ langsam sein, wenn die Anzahl der Stichproben wächst. Sie könnten sich fragen, ob diese
Anforderung, alle Daten bei jeder Iteration zu verwenden, gelockert werden kann;
beispielsweise könnten Sie einfach eine Teilmenge der Daten verwenden, um die Clusterzentren
bei jedem Schritt zu aktualisieren.

Dies ist die Idee hinter den batchbasierten k-Means-Algorithmen, von denen eine Form
in `sklearn.cluster.MiniBatchKMeans` implementiert ist. Die Schnittstelle hierfür ist die
gleiche wie für Standard-K-Means.

### Limitiert auf lineare Cluster Grenzen

Die Grenze zwischen zwei Clustern ist immer eine Gerade, daher hat der Algorithmus Probleme, wenn
die Cluster kompliziertere Formen haben:

In [ ]:
from sklearn.datasets import make_moons
from sklearn.cluster import KMeans

X_moon, y = make_moons(200, noise=.05, random_state=0)
kmeans = KMeans(2, random_state=0)
kmeans.fit(X_moon)
y_means = kmeans.predict(X_moon)
centers = kmeans.cluster_centers_
plt.scatter(X_moon[:, 0], X_moon[:, 1], c=y_means, s=50, cmap='viridis')
plt.scatter(centers[:, 0], centers[:, 1], c='red', s=100);


## Alternative Algorithmen

- Gaussian mixture models (bessere Bewertung über Anzahl der Cluster)
- Algorithmen, die Anzahl der Cluster selber wählen können:
    - DBSCAN
    - mean-shift
    - affinity propagation